In [65]:
import altair as alt
from pathlib import Path
import json
import pandas as pd
import numpy as np

alt.renderers.enable('altair_viewer', inline=True)

RendererRegistry.enable('altair_viewer')

In [15]:
source = pd.read_csv('2020_07_28_15_48.csv')
alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='lut:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color='type:O'
)

FileNotFoundError: [Errno 2] No such file or directory: '2020_07_28_15_48.csv'

In [14]:
s

alt.Chart(...)

In [131]:
source = pd.read_csv('7_31.csv')
alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='dsp:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color='type:O'
)


alt.Chart(...)

In [130]:
source = pd.read_csv('7_31.csv')
sim = pd.read_csv('results/2020_07_30_16_46/sim.csv')

source = source[source['type'] != 'hls_total']

for _, row in sim.iterrows():
    bench_filt = source['benchmark'] == row[0]
    type_filt = source['type'] == 'futil'
    source.loc[(bench_filt) & (type_filt), 'latency'] = row[1]

alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='latency:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color='type:O',
    tooltip=['latency']
)

alt.Chart(...)

In [173]:
source = pd.read_csv('7_31.csv')
sim = pd.read_csv('results/2020_07_30_16_46/sim.csv')

source = source[source['type'] != 'hls_total']

for _, row in sim.iterrows():
    bench_filt = source['benchmark'] == row[0]
    type_filt = source['type'] == 'futil'
    source.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
#     print(row[1])

futil = alt.Chart(source[source['type'] == 'futil']).mark_circle(size=100).encode(
    x='lut:Q',
    y='latency:Q',
    color='type:O',
    tooltip=['latency', 'lut', 'benchmark']
)

hls = alt.Chart(source[source['type'] == 'hls']).mark_circle(size=25).encode(
    x='lut:Q',
    y='latency:Q',
    color='type:O',
    tooltip=['latency', 'lut', 'benchmark']
)


base = alt.layer(futil, hls)

for bench in source['benchmark'].unique():
    df = source[source['benchmark'] == bench]
    N = 30
    lut_arr = np.linspace(df[df['type'] == 'futil']['lut'], df[df['type'] == 'hls']['lut'], num=N)
    lat_arr = np.linspace(df[df['type'] == 'futil']['latency'], df[df['type'] == 'hls']['latency'], num=N)
    df.insert(6, 'dist', [0, 11])
    for i in range(N):
        df = df.append({
           'benchmark': bench,
            'type': 'linear',
            'lut': lut_arr[i][0],
            'dsp': 0,
            'latency': lat_arr[i][0],
            'dist': N - i + 5,
            'source': 'n/a'
        }, ignore_index=True)
            
    chart = alt.Chart(df).mark_trail().encode(
        x='lut:Q',
        y='latency:Q',
        size='dist:Q',
        tooltip=['latency', 'lut']
    )
    base = alt.layer(base, chart)

base.interactive()


alt.LayerChart(...)

In [205]:
source = pd.read_csv('7_31.csv')
sim = pd.read_csv('results/2020_07_30_16_46/sim.csv')

# don't care about hls total for this
source = source[source['type'] != 'hls_total']

for _, row in sim.iterrows():
    bench_filt = source['benchmark'] == row[0]
    type_filt = source['type'] == 'futil'
    source.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
    
for bench in source['benchmark'].unique():
    df = source[source['benchmark'] == bench]
    source = source.append({
        'benchmark': bench,
        'type': 'difference',
        'lut': df[df['type'] == 'futil']['lut'].item() - df[df['type'] == 'hls']['lut'].item(),
        'dsp': 0,
        'latency': df[df['type'] == 'futil']['latency'].item() - df[df['type'] == 'hls']['latency'].item(),
        'source': 'n/a'
    }, ignore_index=True)

src = source[source['type'] == 'difference']
# src = source

lut = alt.Chart(src).mark_bar(color='green', opacity=0.5).encode(
    x = 'benchmark',
    y = alt.Y('lut', scale=alt.Scale(type='linear', base=10)),
)
lat = alt.Chart(src).mark_bar(size=6).encode(
    x = alt.X('benchmark'),
    y = alt.Y('latency', scale=alt.Scale(type='linear', base=10))
)

(lat + lut).encode(
    tooltip = ['lut', 'latency', 'benchmark']
)

alt.LayerChart(...)